In [1]:
### IMPORTS ###
import pandas as pd
import numpy as np
import pathlib as pl
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import re

!pip install datasets
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

!pip install transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler
from transformers import TrainingArguments

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

!pip install evaluate
import evaluate

from transformers import Trainer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

PATH = "/content/drive/My Drive/TFM/Data/HuHu/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
### CONJUNTOS DE ENTRENAMIENTO, VALIDACIÓN Y TEST ###
df = pd.read_csv(PATH + "train.csv",  sep=',', on_bad_lines='skip', encoding='utf-8', encoding_errors='ignore')
df = df[['tweet', 'mean_prejudice']]
df = df.rename(columns={"tweet": "Text", "mean_prejudice": "Label"})
df.fillna(" ", inplace=True)

X_train = df['Text']
y_train = df['Label']

X_train, X_aux, y_train, y_aux = train_test_split(X_train, y_train, test_size=0.3, random_state=55)
X_val, X_test, y_val, y_test = train_test_split(X_aux, y_aux, test_size=0.66, random_state=55)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

print('Tamaño conjunto de Entrenamiento:', len(X_train))
print('Tamaño conjunto de Validación:', len(X_val))
print('Tamaño conjunto de Evaluación:', len(X_test))

Tamaño conjunto de Entrenamiento: 1869
Tamaño conjunto de Validación: 272
Tamaño conjunto de Evaluación: 530


# Preprocesamiento y Limpieza de Texto

In [4]:
### LIMPIEZA DE TEXTOS ###
stopwords_es = stopwords.words("spanish")
def clean_text(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=word_tokenize(text)
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwords_es]
    # usar los stems
    tokens = [PorterStemmer().stem(word) for word in tokens]
    # eliminamos las palabras con menos de 2 caráceres
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    min_length = 2
    p = re.compile('^[a-zA-Z]+$');
    filtered_tokens=""
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens = filtered_tokens + str(token) + " "

    if filtered_tokens == "":
        return "nada"
    else:
        return filtered_tokens

for index in X_train.index:
    X_train[index] = clean_text(X_train[index])

# Label Encoding

No es necesario hacer Label Encoding para tareas de regresión

# Crear Diccionario

In [5]:
d = {'train':Dataset.from_dict({'Label':y_train, 'Text':X_train}),
     'val':Dataset.from_dict({'Label':y_val, 'Text':X_val}),
     'test':Dataset.from_dict({'Label':y_test, 'Text':X_test})
     }

dict_dataset = DatasetDict(d)

# Tokenization

In [6]:
model_name ='dccuchile/distilbert-base-spanish-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

MAX_LENGTH = 55

In [7]:
def tokenize(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)


data_encodings = dict_dataset.map(tokenize, batched=True)
data_encodings

Map:   0%|          | 0/1869 [00:00<?, ? examples/s]

Map:   0%|          | 0/272 [00:00<?, ? examples/s]

Map:   0%|          | 0/530 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Label', 'Text', 'input_ids', 'attention_mask'],
        num_rows: 1869
    })
    val: Dataset({
        features: ['Label', 'Text', 'input_ids', 'attention_mask'],
        num_rows: 272
    })
    test: Dataset({
        features: ['Label', 'Text', 'input_ids', 'attention_mask'],
        num_rows: 530
    })
})

# Modelo

Necesitamos hacer algunas modificaciones para preparar el dataset para el modelo:

- Eliminamos la columna 'text', porque es un campo que el modelo no espera.
- Renombramos 'label' a 'labels', porque es el nombre que espera el modelo.
- El dataset debe devolver un objeto Tensor en lugar de una lista.

Para pasarle los datos al modelo debemos guardarlos en objetos DataLoader

In [8]:
data_encodings = data_encodings.remove_columns('Text')
data_encodings = data_encodings.rename_column('Label', 'labels')
data_encodings.set_format("torch")
data_encodings

train_dataloader = DataLoader(data_encodings['train'], shuffle=True, batch_size=8)
val_dataloader = DataLoader(data_encodings['val'], batch_size=8)

Para definir el modelo, hay que establecer el número de etiquetas:

In [9]:
NUM_LABELS = 1
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)

Some weights of the model checkpoint at dccuchile/distilbert-base-spanish-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/distilbert-base-spanish-uncased and are newly initialized: ['classifier.bias', 

Como optimizador vamos a utilizar el optijizador Adam, implementado en AdamW, y definimos el learning rate:

In [10]:
optimizer = AdamW(model.parameters(), lr=5e-5)

# Hyperparámetros

In [11]:
#!pip uninstall -y accelerate
!pip install accelerate
args = TrainingArguments(output_dir="./outputs")
args.evaluation_strategy="epoch"
args.per_device_train_batch_size = 32
args.per_device_eval_batch_size = 32
args.num_train_epochs = 5

# Métrica

In [12]:
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error
from scipy import stats

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    # loss metrics
    mse = mean_squared_error(labels, logits)
    rmse = mean_squared_error(labels, logits, squared=False)
    mae = mean_absolute_error(labels, logits)
    smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)
    # performance metrics
    r2 = r2_score(labels, logits)
    # pearson=stats.pearsonr(labels, logits)[0]

    # we return a dictionary with all metrics
    # return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape, "pearson": pearson}
    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape}

# Training

In [13]:
trainer = Trainer(
    model=model,            # modelo que será ajustado
    args = args,     # hiperparámetros
    train_dataset=data_encodings['train'], # conjunto training
    eval_dataset=data_encodings['val'],   # conjunto de validación
    compute_metrics=compute_metrics,    # función para computar las métricas
)

In [14]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Smape
1,No log,0.536287,0.536287,0.732316,0.593397,0.096157,20.740159
2,No log,0.565389,0.565389,0.751924,0.592967,0.047109,20.612213
3,No log,0.631982,0.631982,0.794973,0.629609,-0.065123,21.718222
4,No log,0.746206,0.746206,0.863832,0.684262,-0.257633,23.222337
5,No log,0.783544,0.783544,0.885180,0.699784,-0.320563,23.577682


TrainOutput(global_step=295, training_loss=0.5904668258408369, metrics={'train_runtime': 26.4469, 'train_samples_per_second': 353.35, 'train_steps_per_second': 11.154, 'total_flos': 132976009816650.0, 'train_loss': 0.5904668258408369, 'epoch': 5.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.7835443019866943,
 'eval_mse': 0.7835441827774048,
 'eval_rmse': 0.8851802945137024,
 'eval_mae': 0.6997842192649841,
 'eval_r2': -0.3205625578062852,
 'eval_smape': 23.57768159754136,
 'eval_runtime': 0.2777,
 'eval_samples_per_second': 979.543,
 'eval_steps_per_second': 32.411,
 'epoch': 5.0}

# Evaluación

In [16]:
def get_prediction(text):
    inputs = tokenizer(text, max_length=MAX_LENGTH, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    return outputs[0].item()

y_pred=[get_prediction(text) for text in dict_dataset['test']['Text']]

In [17]:
def print_metrics(y_test, y_pred, lang=''):
    if lang:
        pass
    else:
        #         print("Final results on the whole test dataset")
        print('|   |MSE|RMSE|MAE|R2|SMAPE|PEARSON|')
        print('|---|---|---|---|---|---|---|')

    mse = mean_squared_error(y_test, y_pred)
    # print ("MSE: ", "{:.2f}".format(mse), end=', ')

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    # print("RMSE: ", "{:.2f}".format(rmse), end=', ')

    mae = mean_absolute_error(y_test, y_pred)
    # print("MAE: ", "{:.2f}".format(mae), end=', ')

    r2 = r2_score(y_test, y_pred)
    # print("R2: ", "{:.2f}".format(r2), end=', ')

    diff=[label-pred for (label,pred) in zip(y_test,y_pred)]
    smape= 1/len(y_test) * np.sum(2 * np.abs(diff) / (np.abs(y_test) + np.abs(y_pred))*100)
    # print("SMAPE: ", "{:.2f}".format(smape), end=', ')

    pearson=stats.pearsonr(y_test, y_pred)[0]
    # print("PEARSON: ", "{:.2f}".format(pearson))
    # print()

    text_table=str('|')+lang+str('|') +"{:.2f}".format(mse) \
                +str('|')+"{:.2f}".format(rmse) \
                +str('|')+"{:.2f}".format(mae) \
                +str('|')+"{:.2f}".format(r2) \
                +str('|')+"{:.2f}".format(smape) \
                +str('|')+"{:.2f}".format(pearson) + str('|')

    print(text_table)

In [18]:
print_metrics(y_test, y_pred)

|   |MSE|RMSE|MAE|R2|SMAPE|PEARSON|
|---|---|---|---|---|---|---|
||0.82|0.91|0.72|-0.24|23.89|0.39|
